### Parámetros y referencias

In [0]:
# [04][1] Parámetros y referencias de tablas/rutas
dbutils.widgets.text("CATALOGO",               "workspace")
dbutils.widgets.text("ESQUEMA_PLATA",          "silver_mb")
dbutils.widgets.text("ESQUEMA_ORO",            "gold_mb")
dbutils.widgets.text("PREFIJO_TABLA",          "mb_")
dbutils.widgets.text("VOLUMEN_CATALOGO",       "workspace")
dbutils.widgets.text("VOLUMEN_ESQUEMA",        "default")
dbutils.widgets.text("VOLUMEN_LANDING",        "landing")
dbutils.widgets.text("VOLUMEN_VALIDACION",     "validation")

CATALOGO     = dbutils.widgets.get("CATALOGO")
ESQ_PLATA    = dbutils.widgets.get("ESQUEMA_PLATA")
ESQ_ORO      = dbutils.widgets.get("ESQUEMA_ORO")
PREFIJO      = dbutils.widgets.get("PREFIJO_TABLA")
V_CAT        = dbutils.widgets.get("VOLUMEN_CATALOGO")
V_ESQ        = dbutils.widgets.get("VOLUMEN_ESQUEMA")
V_LAND       = dbutils.widgets.get("VOLUMEN_LANDING")
V_VAL        = dbutils.widgets.get("VOLUMEN_VALIDACION")

T_PLATA   = f"{CATALOGO}.{ESQ_PLATA}.{PREFIJO}events_silver"
T_RUNNING = f"{CATALOGO}.{ESQ_ORO}.{PREFIJO}running_stats"
T_LOG     = f"{CATALOGO}.{ESQ_ORO}.{PREFIJO}ingestion_log"

RUTA_LANDING = f"/Volumes/{V_CAT}/{V_ESQ}/{V_LAND}"
RUTA_VALID   = f"/Volumes/{V_CAT}/{V_ESQ}/{V_VAL}"

display({"T_PLATA": T_PLATA, "T_RUNNING": T_RUNNING, "T_LOG": T_LOG, "RUTA_LANDING": RUTA_LANDING, "RUTA_VALID": RUTA_VALID})


### Estadísticas en ejecución

In [0]:
# [04][2] Running stats (ANTES de validation.csv)
display(spark.sql(f"""
SELECT row_count,
       sum_price,
       CASE WHEN row_count>0 THEN sum_price/row_count ELSE NULL END AS avg_price,
       min_price, max_price, min_ts, max_ts, updated_at
FROM {T_RUNNING}
WHERE id=1
"""))


### Verificación independiente sobre Plata

In [0]:
# [04][3] Verificación con full-scan en Plata (ANTES)
# Nota: esto NO se usa para actualizar stats; solo para comprobar consistencia.
display(spark.sql(f"""
SELECT COUNT(*) AS rows,
       AVG(price) AS avg_price,
       MIN(price) AS min_price,
       MAX(price) AS max_price
FROM {T_PLATA}
"""))


### Enviar validation.csv por el pipeline

In [0]:
# [04][4] Copiar validation.csv → landing (dispara micro-batch adicional en siguiente corrida 01→02→03)
src = f"{RUTA_VALID}/validation.csv"
dst = f"{RUTA_LANDING}/validation.csv"
dbutils.fs.cp(src, dst)
print(f"[INFO] Copiado: {src} → {dst}. Ahora vuelve a ejecutar 01 → 02 → 03 para procesar este archivo.")


### Estadísticas en ejecución (DESPUÉS de procesar validation.csv)

In [0]:
# [04][5] Running stats (DESPUÉS de validation.csv)
# Ejecuta esta celda después de volver a correr 01→02→03 para que el archivo ya esté en Oro.
display(spark.sql(f"""
SELECT row_count,
       sum_price,
       CASE WHEN row_count>0 THEN sum_price/row_count ELSE NULL END AS avg_price,
       min_price, max_price, min_ts, max_ts, updated_at
FROM {T_RUNNING}
WHERE id=1
"""))


### Verificación Plata (DESPUÉS) + auditoría de micro-batches

In [0]:
# [04][6] Verificación Plata (DESPUÉS) + auditoría de micro-batches en T_LOG
display(spark.sql(f"""
SELECT COUNT(*) AS rows,
       AVG(price) AS avg_price,
       MIN(price) AS min_price,
       MAX(price) AS max_price
FROM {T_PLATA}
"""))

display(spark.sql(f"SELECT * FROM {T_LOG} ORDER BY processed_at"))
